In [1]:
import torch
import torch.nn as nn

In [2]:

class ResBlock(nn.Module):
    def __init__(self, n_feat, kernel_size, bias=True, bn=False, act=nn.ReLU(True), res_scale=1):
        super(ResBlock, self).__init__()

        m = []
        for i in range(2):
            m.append(nn.Conv2d(n_feat, n_feat, kernel_size, padding=(kernel_size // 2), bias=bias))
            if bn:
                m.append(nn.BatchNorm2d(n_feat))
            if i == 0:
                m.append(act)

        self.body = nn.Sequential(*m)
        self.res_scale = res_scale

In [3]:
import torch
import torch.nn as nn


class ResBlock(nn.Module):
    def __init__(self, n_feat, kernel_size, bias=True, bn=False, act=nn.ReLU(True), res_scale=1):
        super(ResBlock, self).__init__()

        m = []
        for i in range(2):
            m.append(nn.Conv2d(n_feat, n_feat, kernel_size, padding=(kernel_size // 2), bias=bias))
            if bn:
                m.append(nn.BatchNorm2d(n_feat))
            if i == 0:
                m.append(act)

        self.body = nn.Sequential(*m)
        self.res_scale = res_scale

    def forward(self, x):  # x(n_feat) -> res(n_feat)
        res = self.body(x).mul(self.res_scale)
        res += x
        return res


class EncodingBlock(nn.Module):
    def __init__(self, ch_in):
        super(EncodingBlock, self).__init__()

        body = [
            nn.Conv2d(ch_in, 64, kernel_size=3, padding=3 // 2),
            nn.ReLU(),
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
            nn.Conv2d(64, 128, kernel_size=3, padding=3 // 2)
        ]
        self.body = nn.Sequential(*body)
        self.down = nn.Conv2d(128, 64, kernel_size=3, stride=2, padding=3 // 2)
        self.act = nn.ReLU()

    def forward(self, input):  # input -> f_e(128),down(64)
        f_e = self.body(input)
        down = self.act(self.down(f_e))
        return f_e, down


class EncodingBlockEnd(nn.Module):
    def __init__(self, ch_in):
        super(EncodingBlockEnd, self).__init__()

        head = [
            nn.Conv2d(in_channels=ch_in, out_channels=64, kernel_size=3, padding=3 // 2),
            nn.ReLU()
        ]
        body = [
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),

            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),

            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),

            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),

            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),

            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
        ]
        tail = [
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=3 // 2)
        ]
        self.head = nn.Sequential(*head)
        self.body = nn.Sequential(*body)
        self.tail = nn.Sequential(*tail)

    def forward(self, input):  # input -> f_e(128)
        out = self.head(input)
        f_e = self.body(out) + out
        f_e = self.tail(f_e)
        return f_e


class DecodingBlock(nn.Module):
    def __init__(self, ch_in):
        super(DecodingBlock, self).__init__()

        body = [
            nn.Conv2d(in_channels=ch_in, out_channels=64, kernel_size=3, padding=3 // 2),
            nn.ReLU(),
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, padding=1 // 2)
        ]

        self.up = nn.ConvTranspose2d(128, 128, kernel_size=3, stride=2, padding=1)
        self.act = nn.ReLU()
        self.body = nn.Sequential(*body)

    def forward(self, input, map):  # input(128),map(128) -> out(256)
        # 保证逆向卷积出来的shape和map一致
        up = self.up(input, output_size=[input.shape[0], input.shape[1], map.shape[2], map.shape[3]])
        up = self.act(up)
        out = torch.cat((up, map), 1)  # 在channel 纬度上
        out = self.body(out)
        return out


class DecodingBlockEnd(nn.Module):
    def __init__(self, ch_in):
        super(DecodingBlockEnd, self).__init__()

        body = [
            nn.Conv2d(ch_in, 64, kernel_size=3, padding=3 // 2),
            nn.ReLU(),
            ResBlock(n_feat=64, kernel_size=3),
            ResBlock(n_feat=64, kernel_size=3),
        ]

        self.up = nn.ConvTranspose2d(128, 128, kernel_size=3, stride=2, padding=1)
        self.act = nn.ReLU()
        self.body = nn.Sequential(*body)

    def forward(self, input, map):  # input(128),map(128) -> out(64)
        # 保证逆向卷积出来的shape和map一致
        up = self.up(input, output_size=[input.shape[0], input.shape[1], map.shape[2], map.shape[3]])
        out = self.act(up)
        out = torch.cat((out, map), 1)  # 在channel 纬度上
        out = self.body(out)
        return out


In [4]:
Encoding_block1 = EncodingBlock(64)

In [6]:
Encoding_block1.eval()

EncodingBlock(
  (body): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): ResBlock(
      (body): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (3): ResBlock(
      (body): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (down): Conv2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (act): ReLU()
)

In [5]:
x = torch.randn(1,64,128,128)

In [6]:
import numpy as np
import scipy
from scipy import signal as signal
import torch
import torch.nn as nn


class CandyNet(nn.Module):
    def __init__(self, threshold=10.0, use_cuda=False):
        super(CandyNet, self).__init__()

        self.threshold = threshold
        self.use_cuda = use_cuda

        filter_size = 5
        generated_filters = signal.gaussian(filter_size, std=1.0).reshape([1, filter_size])

        self.gaussian_filter_horizontal = nn.Conv2d(1, 1, kernel_size=(1, filter_size), padding=(0, filter_size // 2))
        self.gaussian_filter_horizontal.weight.data.copy_(torch.from_numpy(generated_filters))
        self.gaussian_filter_horizontal.bias.data.copy_(torch.from_numpy(np.array([0.0])))

        self.gaussian_filter_vertical = nn.Conv2d(1, 1, kernel_size=(filter_size, 1), padding=(filter_size // 2, 0))
        self.gaussian_filter_vertical.weight.data.copy_(torch.from_numpy(generated_filters.T))
        self.gaussian_filter_vertical.bias.data.copy_(torch.from_numpy(np.array([0.0])))

        sobel_filter = np.array([[1, 0, -1],
                                 [2, 0, -2],
                                 [1, 0, -1]])

        self.sobel_filter_horizontal = nn.Conv2d(1, 1, kernel_size=sobel_filter.shape,
                                                 padding=sobel_filter.shape[0] // 2)
        self.sobel_filter_horizontal.weight.data.copy_(torch.from_numpy(sobel_filter))
        self.sobel_filter_horizontal.bias.data.copy_(torch.from_numpy(np.array([0.0])))

        self.sobel_filter_vertical = nn.Conv2d(1, 1, kernel_size=sobel_filter.shape, padding=sobel_filter.shape[0] // 2)
        self.sobel_filter_vertical.weight.data.copy_(torch.from_numpy(sobel_filter.T))
        self.sobel_filter_vertical.bias.data.copy_(torch.from_numpy(np.array([0.0])))

        # filters were flipped manually
        filter_0 = np.array([[0, 0, 0],
                             [0, 1, -1],
                             [0, 0, 0]])

        filter_45 = np.array([[0, 0, 0],
                              [0, 1, 0],
                              [0, 0, -1]])

        filter_90 = np.array([[0, 0, 0],
                              [0, 1, 0],
                              [0, -1, 0]])

        filter_135 = np.array([[0, 0, 0],
                               [0, 1, 0],
                               [-1, 0, 0]])

        filter_180 = np.array([[0, 0, 0],
                               [-1, 1, 0],
                               [0, 0, 0]])

        filter_225 = np.array([[-1, 0, 0],
                               [0, 1, 0],
                               [0, 0, 0]])

        filter_270 = np.array([[0, -1, 0],
                               [0, 1, 0],
                               [0, 0, 0]])

        filter_315 = np.array([[0, 0, -1],
                               [0, 1, 0],
                               [0, 0, 0]])

        all_filters = np.stack(
            [filter_0, filter_45, filter_90, filter_135, filter_180, filter_225, filter_270, filter_315])

        self.directional_filter = nn.Conv2d(1, 8, kernel_size=filter_0.shape, padding=filter_0.shape[-1] // 2)
        self.directional_filter.weight.data.copy_(torch.from_numpy(all_filters[:, None, ...]))
        self.directional_filter.bias.data.copy_(torch.from_numpy(np.zeros(shape=(all_filters.shape[0],))))

    def forward(self, img):  # (batch,channel,height, width)

        batch = img.shape[0]
        img_r = img[:, 0:1]  # batch,1,height, width
        img_g = img[:, 1:2]  # batch,1,height, width
        img_b = img[:, 2:3]  # batch,1,height, width

        blur_horizontal = self.gaussian_filter_horizontal(img_r)  # batch,1,height,width
        blurred_img_r = self.gaussian_filter_vertical(blur_horizontal)  # batch,1,height,width
        blur_horizontal = self.gaussian_filter_horizontal(img_g)  # batch,1,height,width
        blurred_img_g = self.gaussian_filter_vertical(blur_horizontal)  # batch,1,height,width
        blur_horizontal = self.gaussian_filter_horizontal(img_b)  # batch,1,height,width
        blurred_img_b = self.gaussian_filter_vertical(blur_horizontal)  # batch,1,height,width

        blurred_img = torch.stack([blurred_img_r, blurred_img_g, blurred_img_b], dim=1)  # batch,1,height,width
        blurred_img = torch.stack([torch.squeeze(blurred_img)])  # batch,1,height,width

        grad_x_r = self.sobel_filter_horizontal(blurred_img_r)  # batch,1,height,width
        grad_y_r = self.sobel_filter_vertical(blurred_img_r)  # batch,1,height,width
        grad_x_g = self.sobel_filter_horizontal(blurred_img_g)  # batch,1,height,width
        grad_y_g = self.sobel_filter_vertical(blurred_img_g)  # batch,1,height,width
        grad_x_b = self.sobel_filter_horizontal(blurred_img_b)  # batch,1,height,width
        grad_y_b = self.sobel_filter_vertical(blurred_img_b)  # batch,1,height,width

        # COMPUTE THICK EDGES
        grad_mag = torch.sqrt(grad_x_r ** 2 + grad_y_r ** 2)  # batch,1,height,width
        grad_mag += torch.sqrt(grad_x_g ** 2 + grad_y_g ** 2)  # batch,1,height,width
        grad_mag += torch.sqrt(grad_x_b ** 2 + grad_y_b ** 2)  # batch,1,height,width
        grad_orientation = (  # batch,1,height,width
                torch.atan2(grad_y_r + grad_y_g + grad_y_b, grad_x_r + grad_x_g + grad_x_b) * (180.0 / 3.14159))
        grad_orientation += 180.0  # batch,1,height,width
        grad_orientation = torch.round(grad_orientation / 45.0) * 45.0  # batch,1,height,width

        # THIN EDGES (NON-MAX SUPPRESSION)

        all_filtered = self.directional_filter(grad_mag)  # batch,8,height,width
        inidices_positive = (grad_orientation / 45) % 8  # batch,1,height,width
        inidices_negative = ((grad_orientation / 45) + 4) % 8  # batch,1,height,width

        height = inidices_positive.size()[2]
        width = inidices_positive.size()[3]
        pixel_count = height * width

        pixel_range = torch.FloatTensor([range(pixel_count)])  # batch,pixel_range
        if self.use_cuda:
            pixel_range = torch.cuda.FloatTensor([range(pixel_count)])

        indices = (  # batch,pixel_range
                inidices_positive.view(
                    inidices_positive.shape[0],
                    pixel_count).data * pixel_count + pixel_range)

        channel_select_filtered_positive = torch.ones(batch, 1, height, width)  # batch, 1, height, width
        for i in range(batch):
            channel_select_filtered_positive_temp = all_filtered[i].view(-1)[indices[i].long()].view(1, height, width)
            channel_select_filtered_positive[i] = channel_select_filtered_positive_temp

        indices = (  # batch,pixel_range
                inidices_negative.view(
                    inidices_negative.shape[0],
                    pixel_count).data * pixel_count + pixel_range)

        channel_select_filtered_negative = torch.ones(batch, 1, height, width)  # batch, 1, height, width
        for i in range(batch):
            channel_select_filtered_negative_temp = all_filtered[i].view(-1)[indices[i].long()].view(1, height, width)
            channel_select_filtered_negative[i] = channel_select_filtered_negative_temp

        channel_select_filtered = torch.stack(  # batch, 2, height, width
            [channel_select_filtered_positive, channel_select_filtered_negative], dim=1)

        is_max = channel_select_filtered.min(dim=1)[0] > 0.0

        thin_edges = grad_mag.clone()
        thin_edges[is_max == 0] = 0.0

        # THRESHOLD

        thresholded = thin_edges.clone()
        thresholded[thin_edges < self.threshold] = 0.0

        early_threshold = grad_mag.clone()
        early_threshold[grad_mag < self.threshold] = 0.0

        assert grad_mag.size() == grad_orientation.size() == thin_edges.size() == thresholded.size() == early_threshold.size()

        return thresholded


if __name__ == '__main__':
    CandyNet()


In [8]:
with torch.no_grad():
    torch.onnx.export(
        Encoding_block1,
        x,
        'test.onnx',
        #opset_version=11,
        input_names=['input'],
        output_names=['output']
    )

In [9]:
ls

draw.py    out.log      test.onnx        Untitled2.ipynb
nohup.out  python.log3  Untitled1.ipynb  Untitled.ipynb


In [10]:
import onnx

ModuleNotFoundError: No module named 'onnx'

In [7]:
import torch.nn as nn


class ConvUp(nn.Module):

    def __init__(self, ch_in, up_factor):

        super(ConvUp, self).__init__()

        body = [
            nn.Conv2d(ch_in, 64, kernel_size=3, padding=3 // 2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=3 // 2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=3 // 2),
            nn.ReLU(),
        ]

        if up_factor == 2:
            modules_tail = [
                nn.ConvTranspose2d(64, 64, kernel_size=3, stride=up_factor, padding=1, output_padding=1),
                nn.Conv2d(64, ch_in, 3, padding=3 // 2, bias=True)
            ]
        elif up_factor == 3:
            modules_tail = [
                nn.ConvTranspose2d(64, 64, kernel_size=3, stride=up_factor, padding=0, output_padding=0),
                nn.Conv2d(64, ch_in, 3, padding=3 // 2, bias=True)
            ]

        elif up_factor == 4:
            modules_tail = [
                nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.Conv2d(64, ch_in, 3, padding=3 // 2, bias=True)
            ]

        self.body = nn.Sequential(*body)
        self.tail = nn.Sequential(*modules_tail)

    def forward(self, input):

        output = self.body(input)
        output = self.tail(output)
        return output


class ConvDown(nn.Module):

    def __init__(self, ch_in, up_factor):

        super(ConvDown, self).__init__()

        body = [
            nn.Conv2d(ch_in, 64, kernel_size=3, padding=3 // 2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=3 // 2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=3 // 2),
            nn.ReLU(),
        ]

        if up_factor == 4:
            modules_tail = [
                nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=2),
                nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=2),
                nn.Conv2d(64, ch_in, kernel_size=3, padding=3 // 2, bias=True)
            ]
        elif up_factor == 3:
            modules_tail = [
                nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=up_factor),
                nn.Conv2d(64, ch_in, kernel_size=3, padding=3 // 2, bias=True)
            ]
        elif up_factor == 2:
            modules_tail = [
                nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=up_factor),
                nn.Conv2d(64, ch_in, kernel_size=3, padding=3 // 2, bias=True)
            ]

        self.body = nn.Sequential(*body)
        self.tail = nn.Sequential(*modules_tail)

    def forward(self, input):

        out = self.body(input)
        out = self.tail(out)
        return out


In [8]:
class E_DUN(nn.Module):
    def __init__(self, args):
        super(E_DUN, self).__init__()

        self.channel0 = args.n_colors
        self.up_factor = args.scale[0]
        self.patch_size = args.patch_size
        self.batch_size = int(args.batch_size / args.n_GPUs)

        self.Encoding_block1 = EncodingBlock(64)
        self.Encoding_block2 = EncodingBlock(64)
        self.Encoding_block3 = EncodingBlock(64)
        self.Encoding_block4 = EncodingBlock(64)

        self.Encoding_block_end = EncodingBlockEnd(64)

        self.Decoding_block1 = DecodingBlock(256)
        self.Decoding_block2 = DecodingBlock(256)
        self.Decoding_block3 = DecodingBlock(256)
        self.Decoding_block4 = DecodingBlock(256)

        self.feature_decoding_end = DecodingBlockEnd(256)

        self.act = nn.ReLU()

        self.construction = nn.Conv2d(64, 3, 3, padding=1)

        G0 = 64
        kSize = 3
        T = 4
        self.Fe_e = nn.ModuleList(
            [nn.Sequential(*[
                nn.Conv2d(3, G0, kSize, padding=(kSize - 1) // 2, stride=1),
                nn.Conv2d(G0, G0, kSize, padding=(kSize - 1) // 2, stride=1)
            ]) for _ in range(T)])

        self.RNNF = nn.ModuleList(
            [nn.Sequential(*[
                nn.Conv2d((i + 2) * G0, G0, 1, padding=0, stride=1),
                nn.Conv2d(G0, G0, kSize, padding=(kSize - 1) // 2, stride=1),
                self.act,
                nn.Conv2d(64, 3, 3, padding=1)
            ]) for i in range(T)])

        self.Fe_f = nn.ModuleList(
            [nn.Sequential(*[
                nn.Conv2d((2 * i + 3) * G0, G0, 1, padding=0, stride=1)
            ]) for i in range(T - 1)])

        self.eta = nn.ParameterList([nn.Parameter(torch.tensor(0.5)) for _ in range(T)])
        self.delta = nn.ParameterList([nn.Parameter(torch.tensor(0.1)) for _ in range(T)])

        self.conv_up = ConvUp(3, self.up_factor)
        self.conv_down = ConvDown(3, self.up_factor)

        self.candy = CandyNet(3).eval()  # candy算子不需要迭代内部系数

    def forward(self, y):  # [batch_size ,3 ,7 ,270 ,480] ;
        fea_list = []
        V_list = []
        outs = []

        x_texture = torch.nn.functional.interpolate(
            y, scale_factor=self.up_factor, mode='bilinear', align_corners=False)
        x_edge = self.candy(x_texture)
        x = x_edge + x_texture  # 这里可以增加一些倍数，直接相加可能会存在问题

        for i in range(len(self.Fe_e)):
            # --------------------denoising module------------------------
            fea = self.Fe_e[i](x_texture)
            fea_list.append(fea)
            if i != 0:
                fea = self.Fe_f[i - 1](torch.cat(fea_list, 1))
            encode0, down0 = self.Encoding_block1(fea)
            encode1, down1 = self.Encoding_block2(down0)
            encode2, down2 = self.Encoding_block3(down1)
            encode3, down3 = self.Encoding_block4(down2)

            media_end = self.Encoding_block_end(down3)

            decode3 = self.Decoding_block1(media_end, encode3)
            decode2 = self.Decoding_block2(decode3, encode2)
            decode1 = self.Decoding_block3(decode2, encode1)
            decode0 = self.feature_decoding_end(decode1, encode0)

            fea_list.append(decode0)
            V_list.append(decode0)
            if i == 0:
                decode0 = self.construction(self.act(decode0))
            else:
                decode0 = self.RNNF[i - 1](torch.cat(V_list, 1))
            v = x_texture + decode0

            # --------------------texture module--------------------------
            x_texture = x_texture - self.delta[i] * (
                    self.conv_up(self.conv_down(x) - y) + self.eta[i] * (x - v))

            # -----------------------edge module--------------------------
            x_edge = self.candy(x)
            x = x_edge + x_texture  # 这里可以增加一些倍数，直接相加可能会存在问题

            outs.append(x)

        return x


In [11]:
class test:
    def __init__(self):
        self.n_colors = 3
        self.scale = [2]
        self.patch_size = 2
        self.batch_size = 1
        self.n_GPUs = 1

In [12]:
args = test()

In [13]:
model = E_DUN(args)
model.eval()

E_DUN(
  (Encoding_block1): EncodingBlock(
    (body): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): ResBlock(
        (body): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (3): ResBlock(
        (body): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (down): Conv2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (act): ReLU()
  )
  (Encoding_block2): EncodingBlock(
    (body): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [ ]:
x = torch.randn(1,3,256,256)
with torch.no_grad():
    torch.onnx.export(
        model,
        x,
        'test.onnx',
        #opset_version=11,
        input_names=['input'],
        output_names=['output']
    )

/root/anaconda3/envs/aaa/lib/python3.6/site-packages/ipykernel_launcher.py:122: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/ipykernel_launcher.py:132: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/ipykernel_launcher.py:134: TracerWarning: There are 2 live references to the data region being modified when tracing in-place operator copy_ (possibly due to an assignment). This might cause the trace to be incorrect, because all other views that also reference

In [14]:
candy = CandyNet(3)

In [15]:
candy

CandyNet(
  (gaussian_filter_horizontal): Conv2d(1, 1, kernel_size=(1, 5), stride=(1, 1), padding=(0, 2))
  (gaussian_filter_vertical): Conv2d(1, 1, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
  (sobel_filter_horizontal): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (sobel_filter_vertical): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (directional_filter): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [16]:
list(candy.parameters())

[Parameter containing:
 tensor([[[[0.1353, 0.6065, 1.0000, 0.6065, 0.1353]]]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True),
 Parameter containing:
 tensor([[[[0.1353],
           [0.6065],
           [1.0000],
           [0.6065],
           [0.1353]]]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True),
 Parameter containing:
 tensor([[[[ 1.,  0., -1.],
           [ 2.,  0., -2.],
           [ 1.,  0., -1.]]]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True),
 Parameter containing:
 tensor([[[[ 1.,  2.,  1.],
           [ 0.,  0.,  0.],
           [-1., -2., -1.]]]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True),
 Parameter containing:
 tensor([[[[ 0.,  0.,  0.],
           [ 0.,  1., -1.],
           [ 0.,  0.,  0.]]],
 
 
         [[[ 0.,  0.,  0.],
           [ 0.,  1.,  0.],
           [ 0.,  0., -1.]]],
 
 
         [[[ 0.,  0.,  0.],
           [ 0.,  1.,  

In [8]:
for para in candy.parameters():
    para.requires_grad = False

In [10]:
list(candy.parameters())

[Parameter containing:
 tensor([[[[0.1353, 0.6065, 1.0000, 0.6065, 0.1353]]]]),
 Parameter containing:
 tensor([0.]),
 Parameter containing:
 tensor([[[[0.1353],
           [0.6065],
           [1.0000],
           [0.6065],
           [0.1353]]]]),
 Parameter containing:
 tensor([0.]),
 Parameter containing:
 tensor([[[[ 1.,  0., -1.],
           [ 2.,  0., -2.],
           [ 1.,  0., -1.]]]]),
 Parameter containing:
 tensor([0.]),
 Parameter containing:
 tensor([[[[ 1.,  2.,  1.],
           [ 0.,  0.,  0.],
           [-1., -2., -1.]]]]),
 Parameter containing:
 tensor([0.]),
 Parameter containing:
 tensor([[[[ 0.,  0.,  0.],
           [ 0.,  1., -1.],
           [ 0.,  0.,  0.]]],
 
 
         [[[ 0.,  0.,  0.],
           [ 0.,  1.,  0.],
           [ 0.,  0., -1.]]],
 
 
         [[[ 0.,  0.,  0.],
           [ 0.,  1.,  0.],
           [ 0., -1.,  0.]]],
 
 
         [[[ 0.,  0.,  0.],
           [ 0.,  1.,  0.],
           [-1.,  0.,  0.]]],
 
 
         [[[ 0.,  0.,  0.],
   

In [17]:
candy.eval()

CandyNet(
  (gaussian_filter_horizontal): Conv2d(1, 1, kernel_size=(1, 5), stride=(1, 1), padding=(0, 2))
  (gaussian_filter_vertical): Conv2d(1, 1, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0))
  (sobel_filter_horizontal): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (sobel_filter_vertical): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (directional_filter): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [20]:
x = torch.randn(1,3,256,256)
with torch.no_grad():
    torch.onnx.export(
        candy,
        x,
        'candy.onnx',
        #opset_version=11,
        input_names=['input'],
        #output_names=['output']
    )

/root/anaconda3/envs/aaa/lib/python3.6/site-packages/ipykernel_launcher.py:122: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/ipykernel_launcher.py:132: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/ipykernel_launcher.py:134: TracerWarning: There are 2 live references to the data region being modified when tracing in-place operator copy_ (possibly due to an assignment). This might cause the trace to be incorrect, because all other views that also reference

RuntimeError: ONNX export failed: Couldn't export operator aten::atan2

Defined at:
<ipython-input-8-6c7c0dfadad2>(108): forward
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/torch/nn/modules/module.py(477): _slow_forward
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/torch/nn/modules/module.py(487): __call__
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/torch/jit/__init__.py(252): forward
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/torch/nn/modules/module.py(489): __call__
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/torch/jit/__init__.py(197): get_trace_graph
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/torch/onnx/utils.py(192): _trace_and_get_graph_from_model
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/torch/onnx/utils.py(224): _model_to_graph
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/torch/onnx/utils.py(281): _export
/root/anaconda3/envs/aaa/lib/python3.6/site-packages/torch/onnx/utils.py(104): export
/root/anaconda3/

In [19]:
candy(x)

tensor([[[[ 0.0000, 25.9050,  0.0000,  ...,  0.0000,  0.0000, 17.5363],
          [ 0.0000,  0.0000, 31.3237,  ..., 27.4972,  0.0000, 19.5156],
          [30.4530, 40.9091,  0.0000,  ...,  0.0000, 17.6542,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, 17.8266],
          [24.5876, 30.4273, 34.9075,  ...,  0.0000, 20.9378, 15.7724],
          [23.1771,  0.0000,  0.0000,  ...,  0.0000, 14.5815,  0.0000]]]],
       grad_fn=<IndexPutBackward>)

In [22]:
model = ConvDown(3,2)

In [23]:
x.shape

torch.Size([1, 3, 256, 256])

In [25]:
model.eval()

ConvDown(
  (body): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
  )
  (tail): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)

In [27]:
with torch.no_grad():
    torch.onnx.export(
        model,
        x,
        'ConvDown.onnx',
        #opset_version=11,
        input_names=['input'],
        output_names=['output']
    )

In [9]:
model = ConvUp(3,2)

In [10]:
x = torch.randn(1,3,128,128)

In [11]:
with torch.no_grad():
    torch.onnx.export(
        model,
        x,
        'ConvUp.onnx',
        #opset_version=11,
        input_names=['input'],
        output_names=['output']
    )